In [1]:
import pandas as pd
import numpy as np
import os
import gc
import torch


In [2]:
# !pip install -r requirements.txt

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "xxx"

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
df = pd.read_csv('Input_Data/test_subsampled.csv')
df.head()

,_id,answer,question,supporting_facts,context,type,level
0,5abc0a5d5542993f40c73c64,no,Are Freakonomics and In the Realm of the Hacke...,"[['Freakonomics (film)', 0], ['In the Realm of...","[['Hackers (film)', ['Hackers is a 1995 Americ...",comparison,hard
1,5a760ab65542994ccc918697,Nelson Rockefeller,Alfred Balk served as the secretary of the Com...,"[['Alfred Balk', 0], ['Alfred Balk', 2], ['Nel...","[['Cynthia Hogan', ['Cynthia C. Hogan (born Ci...",bridge,hard
2,5a8a43eb5542996c9b8d5e82,"Marion, South Australia",Which Australian city founded in 1838 contains...,"[['Westminster School, Adelaide', 0], ['Westmi...","[['Sainik School Balachadi, Jamnagar', ['The S...",bridge,hard
3,5ae7ba7a5542993210983f12,Usher,What is the name of the singer who's song was ...,"[[""I Don't Wanna Know"", 1], ['Yeah! (Usher son...","[[""Can't Touch It"", ['""Can\'t Touch It"" is a s...",bridge,hard
4,5aba7cfe554299232ef4a2fd,Carabao Cup,The 2017–18 Wigan Athletic F.C. season will be...,"[['2017–18 Wigan Athletic F.C. season', 1], ['...","[['2017–18 Wigan Athletic F.C. season', [""The ...",bridge,hard


In [6]:
df = df[['question','context','answer']]
df = df.sample(frac=1, random_state=42)
df = df.reset_index(drop=True)
df = pd.DataFrame(df)

In [8]:
import tiktoken
import ast
# Function to count tokens in a string using tiktoken
def count_tokens(text):
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))

# Function to safely parse context - handles both string and list formats
def parse_context(context):
    """
    Parse context whether it's stored as string representation of list or actual list
    """
    if isinstance(context, str):
        try:
            # Try to parse as literal (for string representations of lists)
            parsed = ast.literal_eval(context)
            return parsed
        except (ValueError, SyntaxError):
            # If it's just a plain string, return as is
            return context
    else:
        # If it's already a list or other structure, return as is
        return context

# Function to count tokens for nested context structure
def count_context_tokens(context):
    """
    Count tokens in context, handling different possible formats
    """
    total_tokens = 0
    
    # First parse the context
    parsed_context = parse_context(context)
    
    # Handle different formats
    if isinstance(parsed_context, list):
        for item in parsed_context:
            if isinstance(item, (list, tuple)) and len(item) >= 2:
                # Expected format: [title, text_snippets]
                title = item[0]
                text_snippets = item[1]
                
                # Count tokens in title
                if isinstance(title, str):
                    total_tokens += count_tokens(title)
                
                # Count tokens in text snippets
                if isinstance(text_snippets, list):
                    for snippet in text_snippets:
                        if isinstance(snippet, str):
                            total_tokens += count_tokens(snippet)
                elif isinstance(text_snippets, str):
                    total_tokens += count_tokens(text_snippets)
            else:
                # Handle single items or unexpected formats
                if isinstance(item, str):
                    total_tokens += count_tokens(item)
    elif isinstance(parsed_context, str):
        # If it's just a string, count tokens directly
        total_tokens += count_tokens(parsed_context)
    
    return total_tokens


# Apply the count_context_tokens function to the 'context' column
df['context_token_count'] = df['context'].apply(count_context_tokens)

In [9]:
df

,question,context,answer,context_token_count
0,Cagliostro-Walzer op.370 was composed by a man...,"[['Jiroemon Kimura', ['Jiroemon Kimura (木村 次郎右...",1825,1754
1,What album did John Lennon release before the ...,"[['Instant Karma: All-Time Greatest Hits', [""I...",John Lennon/Plastic Ono Band,1350
2,Which actor in The Lieutenant Wears Skirts is ...,"[['The Lieutenant Wore Skirts', ['The Lieutena...",Tom Ewell,1050
3,"In what year was the Sayrevill, New Jersey roc...","[['The Rats (American band)', ['The Rats were ...",1983,1516
4,"Which genus has the most number of species, My...","[['Styrax crotonoides', ['Styrax crotonoides i...",Myrsine,824
...,...,...,...,...
495,Luke Rockhold defeated the MMA fighter who was...,"[['Kamen Georgiev', ['Kamen Georgiev (Bulgaria...",Anderson Silva,1559
496,"Who was born first, Brooklyn Decker or Andy Ro...","[['Brooklyn Decker', ['Brooklyn Danielle Decke...",Andrew Stephen Roddick,1339
497,Which Outer Banks landmark and premiere BBQ re...,"[[""The Park's Finest"", ['The Park’s Finest is ...",Pigman's Bar-B-Que,1178
498,What is the name of the largest shipping facil...,"[['Transportation in Boston', ['The Boston tra...",Port of Boston,1227


In [10]:
# Find the maximum token count
max_tokens = df['context_token_count'].max()

In [11]:
# Function to flatten context for processing
def flatten_context(context):
    """
    Flatten context into a single string, handling different formats
    """
    parsed_context = parse_context(context)
    
    if isinstance(parsed_context, list):
        flattened_parts = []
        for item in parsed_context:
            if isinstance(item, (list, tuple)) and len(item) >= 2:
                # Expected format: [title, text_snippets]
                title = item[0]
                text_snippets = item[1]
                
                # Add title
                if isinstance(title, str):
                    flattened_parts.append(title)
                
                # Add text snippets
                if isinstance(text_snippets, list):
                    for snippet in text_snippets:
                        if isinstance(snippet, str):
                            flattened_parts.append(snippet)
                elif isinstance(text_snippets, str):
                    flattened_parts.append(text_snippets)
            else:
                # Handle single items
                if isinstance(item, str):
                    flattened_parts.append(item)
        
        return " ".join(flattened_parts)
    elif isinstance(parsed_context, str):
        return parsed_context
    else:
        return str(parsed_context)

df['raw_context'] = df['context'].apply(flatten_context)

In [12]:
# import matplotlib.pyplot as plt
# df.context_token_count.hist(bins=100)
# plt.show()

In [13]:
df

,question,context,answer,context_token_count,raw_context
0,Cagliostro-Walzer op.370 was composed by a man...,"[['Jiroemon Kimura', ['Jiroemon Kimura (木村 次郎右...",1825,1754,"Jiroemon Kimura Jiroemon Kimura (木村 次郎右衛門 , Ki..."
1,What album did John Lennon release before the ...,"[['Instant Karma: All-Time Greatest Hits', [""I...",John Lennon/Plastic Ono Band,1350,Instant Karma: All-Time Greatest Hits Instant ...
2,Which actor in The Lieutenant Wears Skirts is ...,"[['The Lieutenant Wore Skirts', ['The Lieutena...",Tom Ewell,1050,The Lieutenant Wore Skirts The Lieutenant Wore...
3,"In what year was the Sayrevill, New Jersey roc...","[['The Rats (American band)', ['The Rats were ...",1983,1516,The Rats (American band) The Rats were an Amer...
4,"Which genus has the most number of species, My...","[['Styrax crotonoides', ['Styrax crotonoides i...",Myrsine,824,Styrax crotonoides Styrax crotonoides is a spe...
...,...,...,...,...,...
495,Luke Rockhold defeated the MMA fighter who was...,"[['Kamen Georgiev', ['Kamen Georgiev (Bulgaria...",Anderson Silva,1559,Kamen Georgiev Kamen Georgiev (Bulgarian: Каме...
496,"Who was born first, Brooklyn Decker or Andy Ro...","[['Brooklyn Decker', ['Brooklyn Danielle Decke...",Andrew Stephen Roddick,1339,Brooklyn Decker Brooklyn Danielle Decker Roddi...
497,Which Outer Banks landmark and premiere BBQ re...,"[[""The Park's Finest"", ['The Park’s Finest is ...",Pigman's Bar-B-Que,1178,The Park's Finest The Park’s Finest is a Filip...
498,What is the name of the largest shipping facil...,"[['Transportation in Boston', ['The Boston tra...",Port of Boston,1227,Transportation in Boston The Boston transporta...


In [16]:
# Function to create chunks with overlap while preserving sentence boundaries
def create_chunks(text, chunk_size=256, overlap=64):
    # Split text into sentences using newlines
    sentences = text.split('\n')

    chunks = []
    current_chunk = []
    current_length = 0

    i = 0
    while i < len(sentences):
        sentence = sentences[i]
        sentence_length = count_tokens(sentence)

        # If adding this sentence would exceed chunk size, finalize current chunk
        if current_length + sentence_length > chunk_size and current_chunk:
            # Join current chunk sentences
            chunk_text = '\n'.join(current_chunk)
            chunks.append(chunk_text)

            # Create overlap for next chunk
            overlap_chunk = []
            overlap_length = 0

            # Go backwards from current position to create overlap
            j = len(current_chunk) - 1
            while j >= 0 and overlap_length < overlap:
                overlap_sentence = current_chunk[j]
                overlap_sentence_length = count_tokens(overlap_sentence)

                if overlap_length + overlap_sentence_length <= overlap:
                    overlap_chunk.insert(0, overlap_sentence)
                    overlap_length += overlap_sentence_length
                    j -= 1
                else:
                    break

            # Start new chunk with overlap
            current_chunk = overlap_chunk
            current_length = overlap_length

        # Add current sentence to chunk
        current_chunk.append(sentence)
        current_length += sentence_length
        i += 1

    # Add the last chunk if it has content
    if current_chunk:
        chunk_text = '\n'.join(current_chunk)
        chunks.append(chunk_text)

    # Join chunks with '<c>' separator
    return '<c>'.join(chunks)

# Create the chunks column
df['chunks'] = df['raw_context'].apply(create_chunks)

In [7]:
df

,question,context,answer
0,Cagliostro-Walzer op.370 was composed by a man...,"[['Jiroemon Kimura', ['Jiroemon Kimura (木村 次郎右...",1825
1,What album did John Lennon release before the ...,"[['Instant Karma: All-Time Greatest Hits', [""I...",John Lennon/Plastic Ono Band
2,Which actor in The Lieutenant Wears Skirts is ...,"[['The Lieutenant Wore Skirts', ['The Lieutena...",Tom Ewell
3,"In what year was the Sayrevill, New Jersey roc...","[['The Rats (American band)', ['The Rats were ...",1983
4,"Which genus has the most number of species, My...","[['Styrax crotonoides', ['Styrax crotonoides i...",Myrsine
...,...,...,...
495,Luke Rockhold defeated the MMA fighter who was...,"[['Kamen Georgiev', ['Kamen Georgiev (Bulgaria...",Anderson Silva
496,"Who was born first, Brooklyn Decker or Andy Ro...","[['Brooklyn Decker', ['Brooklyn Danielle Decke...",Andrew Stephen Roddick
497,Which Outer Banks landmark and premiere BBQ re...,"[[""The Park's Finest"", ['The Park’s Finest is ...",Pigman's Bar-B-Que
498,What is the name of the largest shipping facil...,"[['Transportation in Boston', ['The Boston tra...",Port of Boston


In [18]:
# MWVodUZSeW5kSmJBV3Fnc19QK0hkZzVWUU1JYW03TFJaUU1COHVsSFRGL20wbnJ6WVhKU292U2g0WU9RPV92MjAw

In [ ]:
from weaviate.classes.config import Property, DataType
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import weaviate

embedder = SentenceTransformer('all-MiniLM-L6-v2')

def embed_text(text: str) -> list[float]:
    return embedder.encode(text).tolist()

client = weaviate.connect_to_weaviate_cloud(
    cluster_url="xxx",
    auth_credentials=weaviate.AuthApiKey(api_key="xxx"),
)

class_name = "VectorRAG"
if not client.collections.exists(class_name):
    client.collections.create(
        name=class_name,
        vectorizer_config=None,  # no auto embedding
        properties=[
            Property(name="content", data_type=DataType.TEXT),
        ],
    )

collection = client.collections.get(class_name)

with collection.batch.dynamic() as batch:
    for chunks_str in tqdm(df["chunks"], desc="Uploading chunks"):
        chunks = [c.strip() for c in str(chunks_str).split("<c>") if c.strip()]
        for chunk in chunks:
            vector = embed_text(chunk)
            batch.add_object(
                properties={"content": chunk},
                vector=vector
            )


2025-10-29 11:35:20.977496: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/nakul/codecommit/.venv/lib/python3.10/site-packages/weaviate/__init__.py:143: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


In [ ]:
def get_top_chunks(question: str, top_k: int = 3):
    vector = embed_text(question)
    response = collection.query.near_vector(
        near_vector=vector,
        limit=top_k,
        return_properties=["content"]
    )
    chunks = [r.properties["content"] for r in response.objects]
    return " ".join(chunks)


In [10]:
# Apply to all rows in df
tqdm.pandas()
df["retrieved_context"] = df["question"].progress_apply(get_top_chunks)


100%|██████████| 500/500 [00:20<00:00, 24.59it/s]


In [11]:
df

,question,context,answer,retrieved_context
0,Cagliostro-Walzer op.370 was composed by a man...,"[['Jiroemon Kimura', ['Jiroemon Kimura (木村 次郎右...",1825,Alessandro Scarlatti Pietro Alessandro Gaspare...
1,What album did John Lennon release before the ...,"[['Instant Karma: All-Time Greatest Hits', [""I...",John Lennon/Plastic Ono Band,Instant Karma: All-Time Greatest Hits Instant ...
2,Which actor in The Lieutenant Wears Skirts is ...,"[['The Lieutenant Wore Skirts', ['The Lieutena...",Tom Ewell,"Haruma Miura Haruma Miura (三浦 春馬 , Miura Harum..."
3,"In what year was the Sayrevill, New Jersey roc...","[['The Rats (American band)', ['The Rats were ...",1983,The Rats (American band) The Rats were an Amer...
4,"Which genus has the most number of species, My...","[['Styrax crotonoides', ['Styrax crotonoides i...",Myrsine,Styrax crotonoides Styrax crotonoides is a spe...
...,...,...,...,...
495,Luke Rockhold defeated the MMA fighter who was...,"[['Kamen Georgiev', ['Kamen Georgiev (Bulgaria...",Anderson Silva,Kamen Georgiev Kamen Georgiev (Bulgarian: Каме...
496,"Who was born first, Brooklyn Decker or Andy Ro...","[['Brooklyn Decker', ['Brooklyn Danielle Decke...",Andrew Stephen Roddick,Brooklyn Decker Brooklyn Danielle Decker Roddi...
497,Which Outer Banks landmark and premiere BBQ re...,"[[""The Park's Finest"", ['The Park’s Finest is ...",Pigman's Bar-B-Que,The Park's Finest The Park’s Finest is a Filip...
498,What is the name of the largest shipping facil...,"[['Transportation in Boston', ['The Boston tra...",Port of Boston,Transportation in Boston The Boston transporta...


In [12]:
df.to_csv('Ouput_Data/VectorRAG_retrieved_k1.csv',index=False)

In [1]:
import pandas as pd
df = pd.read_csv('Ouput_Data/VectorRAG_retrieved_k1.csv')
df

,question,context,answer,retrieved_context
0,Cagliostro-Walzer op.370 was composed by a man...,"[['Jiroemon Kimura', ['Jiroemon Kimura (木村 次郎右...",1825,Alessandro Scarlatti Pietro Alessandro Gaspare...
1,What album did John Lennon release before the ...,"[['Instant Karma: All-Time Greatest Hits', [""I...",John Lennon/Plastic Ono Band,Instant Karma: All-Time Greatest Hits Instant ...
2,Which actor in The Lieutenant Wears Skirts is ...,"[['The Lieutenant Wore Skirts', ['The Lieutena...",Tom Ewell,"Haruma Miura Haruma Miura (三浦 春馬 , Miura Harum..."
3,"In what year was the Sayrevill, New Jersey roc...","[['The Rats (American band)', ['The Rats were ...",1983,The Rats (American band) The Rats were an Amer...
4,"Which genus has the most number of species, My...","[['Styrax crotonoides', ['Styrax crotonoides i...",Myrsine,Styrax crotonoides Styrax crotonoides is a spe...
...,...,...,...,...
495,Luke Rockhold defeated the MMA fighter who was...,"[['Kamen Georgiev', ['Kamen Georgiev (Bulgaria...",Anderson Silva,Kamen Georgiev Kamen Georgiev (Bulgarian: Каме...
496,"Who was born first, Brooklyn Decker or Andy Ro...","[['Brooklyn Decker', ['Brooklyn Danielle Decke...",Andrew Stephen Roddick,Brooklyn Decker Brooklyn Danielle Decker Roddi...
497,Which Outer Banks landmark and premiere BBQ re...,"[[""The Park's Finest"", ['The Park’s Finest is ...",Pigman's Bar-B-Que,The Park's Finest The Park’s Finest is a Filip...
498,What is the name of the largest shipping facil...,"[['Transportation in Boston', ['The Boston tra...",Port of Boston,Transportation in Boston The Boston transporta...


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
device = 'cuda:1'
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1",device_map =device)
llm = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1",device_map = device)

def answer_generator_single_hop(query, context):
    # Use the globally loaded tokenizer and llm from file_context_1
    # Prepare the prompt for the model
    prompt = (
        "You are an expert at answering the question just based on the context. "
        "Given the context, answer the user question. If you cannot answer the question based on context, "
        "state properly that you cannot answer the question.\n\n"
        f"Context:\n{context}\n\nUser question: {query}\nAnswer:"
    )

    # Tokenize and generate, passing attention_mask for reliable results
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    pad_token_id = tokenizer.eos_token_id

    output_ids = llm.generate(input_ids, attention_mask=attention_mask, pad_token_id=pad_token_id)
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract only the answer after "Answer:"
    if "Answer:" in output:
        answer = output.split("Answer:")[-1].strip().split("\n")[0]
    else:
        answer = output.strip().split("\n")[-1]
    answer = answer.replace('"', '').replace("'", "")
    return answer

2025-10-30 05:43:06.177868: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 1 has a total capacity of 44.34 GiB of which 174.62 MiB is free. Process 739802 has 41.11 GiB memory in use. Including non-PyTorch memory, this process has 3.04 GiB memory in use. Of the allocated memory 2.66 GiB is allocated by PyTorch, and 87.92 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

df['predicted_answer_mistral'] = df.apply(
    lambda row: answer_generator_single_hop(row['question'], row['retrieved_context']),
    axis=1
)


In [ ]:

# Delete model and tokenizer
del llm
del tokenizer

# Clear CUDA cache (GPU memory)
torch.cuda.empty_cache()

# Run Python garbage collector (cleans CPU memory)
gc.collect()


13469

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
device = 'cuda:2'
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B",device_map=device)
llm = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B",device_map=device)
def answer_generator_single_hop(query, context):
    # Use the globally loaded tokenizer and llm from file_context_1
    # Prepare the prompt for the model
    prompt = (
        "You are an expert at answering the question just based on the context. "
        "Given the context, answer the user question. If you cannot answer the question based on context, "
        "state properly that you cannot answer the question.\n\n"
        f"Context:\n{context}\n\nUser question: {query}\nAnswer:"
    )

    # Tokenize and generate, passing attention_mask for reliable results
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    pad_token_id = tokenizer.eos_token_id

    output_ids = llm.generate(input_ids, attention_mask=attention_mask, pad_token_id=pad_token_id)
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract only the answer after "Answer:"
    if "Answer:" in output:
        answer = output.split("Answer:")[-1].strip().split("\n")[0]
    else:
        answer = output.strip().split("\n")[-1]
    answer = answer.replace('"', '').replace("'", "")
    return answer

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
df.to_csv("Ouput_Data/Vector_RAG_final_output_mistral_LLama.csv")

In [ ]:

df['predicted_answer_LLama'] = df.apply(
    lambda row: answer_generator_single_hop(row['question'], row['retrieved_context']),
    axis=1
)


In [ ]:
df.to_csv("Ouput_Data/Vector_RAG_final_output_mistral_LLama.csv")

In [ ]:

# Delete model and tokenizer
del llm
del tokenizer

# Clear CUDA cache (GPU memory)
torch.cuda.empty_cache()

# Run Python garbage collector (cleans CPU memory)
gc.collect()


273

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
device = 'cuda:1'
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl",device_map=device)
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl").to(device)


def answer_generator_single_hop(query, context, max_new_tokens=128):
    """
    Generate an answer using a seq2seq model (e.g., FLAN-T5) based on the given context and question.
    """
    # Build the model prompt in instruction format (T5-style)
    prompt = (
        "You are an expert at answering the question based on the context. "
        "If the context does not contain the answer, say so clearly.\n\n"
        f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    )

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)

    # Generate output (T5 generates sequence-to-sequence output)
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        # num_beams=4,            # optional: better quality
        early_stopping=True
    )

    # Decode and clean up
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    answer = output_text.strip().replace('"', '').replace("'", "")

    return answer


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

df['predicted_answer_FlanT5'] = df.apply(
    lambda row: answer_generator_single_hop(row['question'], row['retrieved_context']),
    axis=1
)


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
df.to_csv("Ouput_Data/Vector_RAG_final_output_mistral_LLama_Flant5.csv")

In [ ]:

# Delete model and tokenizer
del llm
del tokenizer

# Clear CUDA cache (GPU memory)
torch.cuda.empty_cache()

# Run Python garbage collector (cleans CPU memory)
gc.collect()
